# LLM Calls

In [4]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
# llm = OpenAI(model="gpt-3.5-turbo")
chat_model = ChatOpenAI(model="gpt-3.5-turbo")
# llm.predict("hi!")
chat_model.predict("Hi")

'Hello! How can I assist you today?'

# Download competitions

In [2]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [8]:
api.dataset_download_file('kaggle/meta-kaggle/download?datasetVersionNumber=1131',
                          file_name='Competitions.csv',
                          path='./data')

api.dataset_download_file('kaggle/meta-kaggle',
                          file_name='ForumTopics.csv',
                          path='./data')

True

In [21]:
import zipfile

def extract_zip(name):
    zip_path = f'./data/{name}.csv.zip'  
    extract_path = './data'  

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

extract_zip("ForumTopics")

In [19]:
import pandas as pd
forums_df = pd.read_csv("./data/ForumTopics.csv")
forums_df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/ForumTopics.csv'

In [3]:
comp_df = pd.read_csv("./data/Competitions.csv")
# we can filter to keep only the best competitions, i.e. "Featured", "Reseach" and recruitments

In [17]:

comp_slug = "Eurovision2010"
comp_url = "https://www.kaggle.com/c/"+str(comp_slug)
comp_url

'https://www.kaggle.com/c/Eurovision2010'

In [52]:
df = pd.read_csv("../data/competitions_with_descriptions.csv")
df.head(5)

level_0  index    Id            Slug  \
0      0.0    0.0  2408  Eurovision2010   
1      1.0    1.0  2435  hivprogression   
2      2.0    2.0  2438    worldcup2010   
3      3.0    3.0  2439     informs2010   
4      4.0    4.0  2442    worldcupconf   

                                   Title  \
0             Forecast Eurovision Voting   
1                Predict HIV Progression   
2    World Cup 2010 - Take on the Quants   
3       INFORMS Data Mining Contest 2010   
4  World Cup 2010 - Confidence Challenge   

                                                                                                                                                                                   Subtitle  \
0                                                This competition requires contestants to forecast the voting for this year's Eurovision Song Contest in Norway on May 25th, 27th and 29th.   
1              This contest requires competitors to predict the likelihood that an HIV patient's infection will become less severe, given a small dataset and limited clinical information.   
2                                                                          Quants at Goldman Sachs and JP Morgan have modeled the likely outcomes of the 2010 World Cup. Can you do better?   
3  The goal of this contest is to predict short term movements in stock prices. The winners of this contest will be honoured of the INFORMS Annual Meeting in Austin-Texas (November 7-10).   
4                                                                             The Confidence Challenge requires competitors to assign a level of confidence to their World Cup predictions.   

  HostSegmentTitle  ForumId  OrganizationId          EnabledDate  \
0         Featured        2             NaN  04/07/2010 07:57:43   
1         Featured        1             NaN  04/27/2010 21:29:09   
2         Featured  3094129             NaN  06/03/2010 08:08:08   
3         Featured        4             NaN  06/21/2010 21:53:25   
4         Featured        3             NaN  06/03/2010 08:08:08   

          DeadlineDate ProhibitNewEntrantsDeadlineDate TeamMergerDeadlineDate  \
0  05/25/2010 18:00:00                             NaN                    NaN   
1  08/02/2010 12:32:00                             NaN                    NaN   
2  06/11/2010 13:29:00                             NaN                    NaN   
3  10/10/2010 02:28:00                             NaN                    NaN   
4  06/11/2010 13:28:00                             NaN                    NaN   

  TeamModelDeadlineDate ModelSubmissionDeadlineDate  \
0                   NaN                         NaN   
1                   NaN                         NaN   
2                   NaN                         NaN   
3                   NaN                         NaN   
4                   NaN                         NaN   

   FinalLeaderboardHasBeenVerified  HasKernels  OnlyAllowKernelSubmissions  \
0                             True        True                       False   
1                             True        True                       False   
2                             True        True                       False   
3                             True        True                       False   
4                             True        True                       False   

   HasLeaderboard  LeaderboardPercentage  LeaderboardDisplayFormat  \
0           False                     10                       0.0   
1            True                     30                       0.0   
2           False                     10                       0.0   
3            True                     10                       0.0   
4           False                     10                       0.0   

  EvaluationAlgorithmAbbreviation  \
0                              AE   
1                             MCE   
2                          Custom   
3                             AUC   
4                          Cus

In [41]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

df["testi"] = ''
df["testi"].head(10)

0    
1    
2    
3    
4    
5    
6    
7    
8    
9    
Name: testi, dtype: object

# Scrapping a competition

In [18]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


options = webdriver.ChromeOptions()
options.add_argument("ignore-certificate-errors")
#options.add_argument("--headless")
driver = webdriver.Chrome(options=options) 

class sibling_content_loaded:
    def __init__(self, sibling_id):
        self.sibling_id = sibling_id
        self.prev_content = None

    def __call__(self, driver):
        sibling = driver.find_element(By.XPATH, f'//*[@id="{self.sibling_id}"]/following-sibling::div[1]')
        curr_content = sibling.get_attribute('innerHTML')
        if self.prev_content == curr_content:
            return True  
        self.prev_content = curr_content
        return False 

def get_page_content(url):
    driver.get(url)

    # element = WebDriverWait(driver, 10).until(
    #     # EC.presence_of_element_located((By.ID, "pageheader-nav-item--overview"))
    #     EC.presence_of_element_located((By.ID, "background"))
    # )
    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'sc-fOVJBw'))) 
    WebDriverWait(driver, 10, poll_frequency=2).until(sibling_content_loaded('abstract'))

    # WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'class_name')))
    # soup = BeautifulSoup(driver.page_source, 'html.parser').find("div", {"id": "background"}).find_parent("div")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    target_div = soup.find('div', id='abstract').find_next_sibling('div')
    return target_div, target_div.get_text()

soup, content = get_page_content('http://www.kaggle.com/c/Eurovision2010')
soup

<div class="sc-fSTSVA hWMDvy"><div class="sc-cWzipB Tlhqc" id="background"><div class="sc-dskThN cGppFq sc-fvjAjA ecCkGb"><div class="sc-cYRmzm SKQgu"><div class="sc-fbHMxb hIxikl" id="Background"><div class="sc-iCEEWV kmJPwG"><h5 class="sc-brKeYL sc-ktEKTO izsmej JpJng">Background</h5></div><button aria-label="Copy link" class="sc-ejdXBC kPmfXR undefined google-material-icons" title="Copy link">link</button></div><button aria-label="Collapse " class="sc-ejdXBC kPmfXR undefined google-material-icons" style="flex-shrink: 0;" title="Collapse ">keyboard_arrow_up</button></div><div><div class="sc-crknUz iZprDw"><div class="sc-dlyeST hCUClb">The Eurovision Song Contest is an annual competition broadcast worldwide which is open to entrants from active members of the European Broadcasting Union (EBU).  This is not defined by Europe's geographical bounds; for example Israel and Cyprus compete in the contest.  This year's contest is being held in Oslo, Norway.  The location is usually the count

In [11]:
content

'Backgroundlinkkeyboard_arrow_upThe Eurovision Song Contest is an annual competition broadcast worldwide which is open to entrants from active members of the European Broadcasting Union (EBU).\xa0 This is not defined by Europe\'s geographical bounds; for example Israel and Cyprus compete in the contest.\xa0 This year\'s contest is being held in Oslo, Norway.\xa0 The location is usually the country of the previous year\'s winner.\xa0 The number of entrants changes on a yearly basis, due to flux in EBU membership, political shifts such as the dissolution of the Former Soviet Union and Yugoslavia, and desire (or lack thereof) of countries to participate.\xa0 \xa0Each country must submit one song to represent it, which must not have been previously commercially released.\xa0 All songs are, however, made public by a deadline several weeks before the contest, and this year\'s entrants are all known already with details such as title, language and artist included in the database provided.\xa0

In [6]:
soup.get_text()

'Citationkeyboard_arrow_upAnthony Goldbloom. (2010). Forecast Eurovision Voting . Kaggle. https://kaggle.com/competitions/Eurovision2010Cite'

# Format competition summary and problem

In [19]:
from langchain.prompts import PromptTemplate

def get_formated_comp_description(text):
    prompt = PromptTemplate.from_template("""Here is data scrapped from kaggle competition. Please format your response this way:
    1. Explain the problem for a machine learning expert
    2. Give the category of the problem
    {competition_text}""")
    return chat_model.predict(prompt.format(competition_text=text))

# Get solutions URL for a competition

In [44]:
def get_discussion_results(forum_id, n):
    results_df = forums_df[forums_df["ForumId"]==forum_id]
    results_df = results_df.sort_values(by=["Score","TotalMessages"], ascending=False).head(n).reset_index(drop=True)
    return results_df[["Title", "Id", "Score", "TotalMessages", "TotalReplies"]]

In [51]:
def get_discussions_urls(comp_id):
    id_forum_comp = comp_df.iloc[comp_id]["ForumId"]
    comp_slug = comp_df.iloc[comp_id]["Slug"]

    discussions_df = get_discussion_results(id_forum_comp, 10)
    url_list = []
    for id_disc, title, score in zip(discussions_df["Id"], discussions_df["Title"], discussions_df["Score"]):
        url = f"https://www.kaggle.com/c/{comp_slug}/discussion/{id_disc}"
        url_list.append((title, url, score))
    
    return url_list

candidate_solutions = get_discussions_urls(5603) 
candidate_solutions


[('Solved, without any submission. This Crab has Eleven Years Kaggling.',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/413750',
  62),
 ('3rd private 6th public | Brute-force ensemble| Post-processing',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/416783',
  62),
 ('Feature engineering that helped me reduce MAE to 1.33708',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/415721',
  49),
 ('Some Domain Materials Regarding the Dataset',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/413741',
  45),
 ('To round or not to round?',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/413971',
  35),
 ('Tips and Tricks to help you improve your score. ( Part 2)',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/415200',
  27),
 ('#1 Solution - Experimenting with the data generation model',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/416766',
  23),
 ("The Most Bizarre Feature Correlat

# Scrapping of URL solution
For a given URL solution, we collect its explanation 

In [53]:
candidate_solutions[3][1]

'https://www.kaggle.com/c/playground-series-s3e16/discussion/413741'

In [15]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def get_solution_scrapping(url):
    options = webdriver.ChromeOptions()
    options.add_argument("ignore-certificate-errors")
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options) 

    driver.get(url)

    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "sc-bZMpAI"))
    )

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup.find(class_="sc-dlyeST")
get_solution_scrapping("https://www.kaggle.com/c/playground-series-s3e16/discussion/413741")

<html lang="en"><head>
<title>Regression with a Crab Age Dataset | Kaggle</title>
<meta charset="utf-8"/>
<meta content="index, follow" name="robots"/>
<meta content="Playground Series - Season 3, Episode 16" name="description"/>
<meta content="no-cache" name="turbolinks-cache-control"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0" name="viewport"/>
<meta content="#008ABC" name="theme-color"/>
<script async="" nonce="" src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.en.L92w_vMR5kE.O/m=auth2,client/rt=j/sv=1/d=1/ed=1/rs=AHpOoo-GjmyR8TQVDKPrbqj5UQg7cU3cCA/cb=gapi.loaded_0?le=scs"></script><script nonce="" type="text/javascript">
    window["pageRequestStartTime"] = 1695655428643;
    window["pageRequestEndTime"] = 1695655428997;
    window["initialPageLoadStartTime"] = new Date().getTime();
  </script>
<link crossorigin="anonymous" href="https://www.google-analytics.com" rel="preconnect"/><link href="https://stats.g.doubleclick.

# Storing and retrieval of embeddings 

In [17]:
soup, content = get_page_content('http://www.kaggle.com/c/Eurovision2010')

In [20]:
formated_comp_desc = get_formated_comp_description(content)

In [21]:
formated_comp_desc

'1. The problem for a machine learning expert is to predict the voting outcomes for the Eurovision Song Contest based on historical voting patterns, betting odds, song details, and artist details. The goal is to accurately predict which countries will make it to the final and how they will vote for each other.\n\n2. The category of the problem is a multi-class classification and regression problem. The classification part involves predicting which countries will make it to the final, while the regression part involves predicting the voting scores for each country.'

In [22]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

def create_document(content):
    return Document(page_content=content, metadata={"source": "local"})

# embeddings = OpenAIEmbeddings.embed_documents(texts=[content])
documents = [create_document(formated_comp_desc), create_document("This is a competition where we have a time serie forecasting problem"), create_document("this is a random string")]

vectorstore = Chroma.from_documents(documents=documents, embedding=OpenAIEmbeddings())

In [33]:
question = "Weather prediction"
docs = vectorstore.similarity_search(question)
docs

[Document(page_content='This is a competition where we have a time serie forecasting problem', metadata={'source': 'local'}),
 Document(page_content='This is a competition where we have a time serie forecasting problem', metadata={'source': 'local'}),
 Document(page_content='1. The problem for a machine learning expert is to predict the voting outcomes for the 2010 Eurovision Song Contest. This involves predicting which 25 countries will make it to the final and then predicting the voting between these countries. The goal is to create a model that accurately predicts the voting results based on historical voting patterns, betting odds, song details, and artist details.\n\n2. The category of the problem is a classification and prediction problem. It involves predicting the categorical outcome of which countries will make it to the final and then predicting the numerical outcome of the voting scores between these countries.', metadata={'source': 'local'}),
 Document(page_content='this is

# Explaining similarity

In [37]:
prompt = PromptTemplate.from_template("""Here is a kaggle competition description {kaggle_desc}. Explain me how this competition is similar to this problem description {problem_desc}. Be concise""")
chat_model.predict(prompt.format(kaggle_desc=docs[0].page_content, problem_desc="Weather prediction"))

'Both the Kaggle competition and the weather prediction problem involve time series forecasting. In both cases, the goal is to predict future values based on historical data.'

In [21]:
import pandas as pd
df = pd.read_csv("../data/competition_with_embeddings.csv")
df.columns

Index(['Id', 'Slug', 'Title', 'Subtitle', 'HostSegmentTitle', 'ForumId',
       'OrganizationId', 'EnabledDate', 'DeadlineDate',
       'ProhibitNewEntrantsDeadlineDate', 'TeamMergerDeadlineDate',
       'TeamModelDeadlineDate', 'ModelSubmissionDeadlineDate',
       'FinalLeaderboardHasBeenVerified', 'HasKernels',
       'OnlyAllowKernelSubmissions', 'HasLeaderboard', 'LeaderboardPercentage',
       'LeaderboardDisplayFormat', 'EvaluationAlgorithmAbbreviation',
       'EvaluationAlgorithmName', 'EvaluationAlgorithmDescription',
       'EvaluationAlgorithmIsMax', 'MaxDailySubmissions',
       'NumScoredSubmissions', 'MaxTeamSize', 'BanTeamMergers',
       'EnableTeamModels', 'RewardType', 'RewardQuantity', 'NumPrizes',
       'UserRankMultiplier', 'CanQualifyTiers', 'TotalTeams',
       'TotalCompetitors', 'TotalSubmissions', 'ValidationSetName',
       'ValidationSetValue', 'EnableSubmissionModelHashes',
       'EnableSubmissionModelAttachments', 'HostName', 'CompetitionTypeId',
      

# Experimenting with FAISS

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

docs = [Document(page_content='This is a competition where we have a time serie forecasting problem', metadata={'source': 'local'}),
 Document(page_content='This is a competition where we have a time serie forecasting problem', metadata={'source': 'local'}),
 Document(page_content='1. The problem for a machine learning expert is to predict the voting outcomes for the 2010 Eurovision Song Contest. This involves predicting which 25 countries will make it to the final and then predicting the voting between these countries. The goal is to create a model that accurately predicts the voting results based on historical voting patterns, betting odds, song details, and artist details.\n\n2. The category of the problem is a classification and prediction problem. It involves predicting the categorical outcome of which countries will make it to the final and then predicting the numerical outcome of the voting scores between these countries.', metadata={'source': 'local'}),
 Document(page_content='this is a random string', metadata={'source': 'local'})]

In [4]:
embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(docs, embeddings)


In [5]:
db.similarity_search("Weather prediction")

[Document(page_content='This is a competition where we have a time serie forecasting problem', metadata={'source': 'local'}),
 Document(page_content='This is a competition where we have a time serie forecasting problem', metadata={'source': 'local'}),
 Document(page_content='1. The problem for a machine learning expert is to predict the voting outcomes for the 2010 Eurovision Song Contest. This involves predicting which 25 countries will make it to the final and then predicting the voting between these countries. The goal is to create a model that accurately predicts the voting results based on historical voting patterns, betting odds, song details, and artist details.\n\n2. The category of the problem is a classification and prediction problem. It involves predicting the categorical outcome of which countries will make it to the final and then predicting the numerical outcome of the voting scores between these countries.', metadata={'source': 'local'}),
 Document(page_content='this is

In [12]:
db.save_local("./data/faiss_index")
new_db = FAISS.load_local("./data/faiss_index", embeddings)
new_db.add_documents(documents=[Document(page_content="d")])
# new_db.similarity_search("Weather prediction", k = 5)
new_db.similarity_search_with_relevance_scores("Weather prediction", k = 5)

[(Document(page_content='This is a competition where we have a time serie forecasting problem', metadata={'source': 'local'}),
  0.7319540092250825),
 (Document(page_content='This is a competition where we have a time serie forecasting problem', metadata={'source': 'local'}),
  0.7318112157023275),
 (Document(page_content='1. The problem for a machine learning expert is to predict the voting outcomes for the 2010 Eurovision Song Contest. This involves predicting which 25 countries will make it to the final and then predicting the voting between these countries. The goal is to create a model that accurately predicts the voting results based on historical voting patterns, betting odds, song details, and artist details.\n\n2. The category of the problem is a classification and prediction problem. It involves predicting the categorical outcome of which countries will make it to the final and then predicting the numerical outcome of the voting scores between these countries.', metadata={'so